GDD Classify Pipeline

1. Step Set the run configuration.
Be sure to set java memory not over your usable RAM
If Metanome fails, be sure to use JAVA Version 18.0.2.1

In [ ]:
from notebook_ui import config_editor
config_editor()

2. Upload a CSV file or a ZIP archive locally. ZIPs will be extracted automatically.

In [ ]:
from notebook_ui import upload_widget
import json

# Load your config
with open("config.json", "r") as f:
    config = json.load(f)

# Display widget
upload_widget(config)

3. Run the classification pipeline on the dataset:

In [3]:
%run test_pipeline.py

-----Starting full classification pipeline with metrics-----
Reading and processing: UserData/realData\achievements.csv


Reading: 100%|██████████| 1/1 [00:00<00:00, 393.65it/s]

Only 78 rows found. Sampling skipped.
Cleaned CSV saved to: UserData/realData\achievements.csv.tmp
Reading and processing: UserData/realData\art.csv


Reading: 100%|██████████| 1/1 [00:00<00:00, 444.12it/s]


Only 12 rows found. Sampling skipped.
Cleaned CSV saved to: UserData/realData\art.csv.tmp
Reading and processing: UserData/realData\bags.csv


...

Only 726 rows found. Sampling skipped.
Cleaned CSV saved to: UserData/realData\bottoms.csv.tmp
Reading and processing: UserData/realData\construction.csv


Reading: 100%|██████████| 1/1 [00:00<00:00, 464.90it/s]

Only 236 rows found. Sampling skipped.
Cleaned CSV saved to: UserData/realData\construction.csv.tmp
[2025-04-03 17:21:44] Metanome result already exists: UserData/realData\metanomeResults\.\achievements_Results.json, skipping.
[2025-04-03 17:21:44] Metanome result already exists: UserData/realData\metanomeResults\.\art_Results.json, skipping.
[2025-04-03 17:21:44] Metanome result already exists: UserData/realData\metanomeResults\.\bags_Results.json, skipping.
[2025-04-03 17:21:44] Metanome result already exists: UserData/realData\metanomeResults\.\bottoms_Results.json, skipping.
[2025-04-03 17:21:44] Metanome result already exists: UserData/realData\metanomeResults\.\construction_Results.json, skipping.
Found 5 datasets in 'UserData/realData'. Starting classification...
UserData/realData\achievements.csv classified as real in 43 ms.
UserData/realData\art.csv classified as real in 47 ms.



F:\VSCode\projects\generatedDatasetDetector\RandomForest\MultipleRandomForestTraining.py:40: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  skew_vals = skew(filtered_numeric_df, nan_policy='omit')
F:\VSCode\projects\generatedDatasetDetector\RandomForest\MultipleRandomForestTraining.py:47: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  features['num_kurtosis'] = np.clip(np.nanmean(kurtosis(filtered_numeric_df, nan_policy='omit')), -10, 10)


UserData/realData\bags.csv classified as real in 51 ms.
UserData/realData\bottoms.csv classified as real in 54 ms.
UserData/realData\construction.csv classified as real in 37 ms.
[2025-04-03 17:21:45] Summary written to UserData/results\run_summary.txt
-----Classification completed-----
